# 🔄 Triangular Arbitrage Bot - Google Colab

Bot de arbitragem triangular para Binance executando no Google Colab.

## 📋 O que este notebook faz?

- Escaneia a Binance buscando oportunidades de arbitragem triangular
- Calcula lucros reais (após taxas)
- Exibe oportunidades em tempo real
- Salva histórico para análise

## ⚡ Início Rápido

1. Execute as células na ordem
2. Configure seu capital (padrão: $60 USD)
3. Deixe o bot escanear
4. Analise os resultados

---

## 🔧 1. Instalação e Setup

In [ ]:
# Instala dependências
!pip install -q python-binance pyyaml pandas plotly matplotlib colorama tabulate

In [ ]:
# Clone o repositório (ou use arquivos locais)
import os

if not os.path.exists('triangular-arbitrage-bot'):
    print("Clonando repositório...")
    !git clone https://github.com/seu-usuario/triangular-arbitrage-bot.git
    %cd triangular-arbitrage-bot
else:
    print("✓ Repositório já existe")
    %cd triangular-arbitrage-bot

## ⚙️ 2. Configuração

In [ ]:
# Configurações do Bot
CAPITAL_BRL = 300.0  # Seu capital em BRL
CAPITAL_USD = 60.0   # Equivalente em USD
MIN_PROFIT_PERCENT = 0.5  # Lucro mínimo (%)
SCAN_INTERVAL = 5  # Segundos entre scans
MODE = 'simulation'  # 'simulation' ou 'live'

print(f"✓ Capital: R$ {CAPITAL_BRL:.2f} (${CAPITAL_USD:.2f})")
print(f"✓ Lucro mínimo: {MIN_PROFIT_PERCENT}%")
print(f"✓ Modo: {MODE.upper()}")

In [ ]:
# Atualiza config.yaml com suas configurações
import yaml

with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

config['capital_brl'] = CAPITAL_BRL
config['capital_usd'] = CAPITAL_USD
config['min_profit_percent'] = MIN_PROFIT_PERCENT
config['update_interval'] = SCAN_INTERVAL
config['mode'] = MODE

with open('config.yaml', 'w') as f:
    yaml.dump(config, f)

print("✓ Configuração atualizada")

## 🚀 3. Executar o Bot

In [ ]:
# Scan único (teste rápido)
print("🔍 Executando scan de teste...\n")
!python main.py --scan-once

In [ ]:
# Scan contínuo por 5 minutos (300 segundos)
print("🔄 Iniciando scan contínuo por 5 minutos...\n")
!python main.py --mode simulation --duration 300

## 📊 4. Análise dos Resultados

In [ ]:
# Carrega oportunidades salvas
import pandas as pd
import plotly.express as px

try:
    df = pd.read_csv('data/opportunities.csv')
    print(f"✓ {len(df)} oportunidades carregadas\n")
    
    # Estatísticas básicas
    print("📊 ESTATÍSTICAS")
    print("="*60)
    print(f"Lucro Total: ${df['profit_usd'].sum():.2f}")
    print(f"Lucro Médio: ${df['profit_usd'].mean():.2f} ({df['profit_percent'].mean():.3f}%)")
    print(f"Melhor: {df['profit_percent'].max():.3f}%")
    print(f"Pior: {df['profit_percent'].min():.3f}%")
    print("="*60)
    
except FileNotFoundError:
    print("⚠️ Nenhum dado encontrado. Execute o bot primeiro.")
    df = None

In [ ]:
# Top 10 oportunidades
if df is not None:
    print("\n🏆 TOP 10 OPORTUNIDADES")
    print("="*80)
    
    top10 = df.nlargest(10, 'profit_percent')
    
    for i, row in top10.iterrows():
        print(f"#{i+1} - Lucro: {row['profit_percent']:.3f}% (${row['profit_usd']:.2f})")
        print(f"    Rota: {row['path']}")
        print()

In [ ]:
# Gráfico: Distribuição de lucros
if df is not None:
    fig = px.histogram(
        df, 
        x='profit_percent',
        nbins=50,
        title='Distribuição de Lucros (%)',
        labels={'profit_percent': 'Lucro (%)', 'count': 'Frequência'}
    )
    fig.show()

In [ ]:
# Gráfico: Lucro acumulado ao longo do tempo
if df is not None and 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp')
    df['cumulative_profit'] = df['profit_usd'].cumsum()
    
    fig = px.line(
        df,
        x='timestamp',
        y='cumulative_profit',
        title='Lucro Acumulado ao Longo do Tempo',
        labels={'timestamp': 'Tempo', 'cumulative_profit': 'Lucro Acumulado (USD)'}
    )
    fig.show()

## 💡 5. Análise Avançada

In [ ]:
# Análise por rota (quais rotas são mais lucrativas)
if df is not None:
    print("\n📈 ANÁLISE POR ROTA\n")
    
    route_analysis = df.groupby('path').agg({
        'profit_percent': ['count', 'mean', 'max'],
        'profit_usd': 'sum'
    }).round(3)
    
    route_analysis.columns = ['Quantidade', 'Lucro Médio %', 'Lucro Máximo %', 'Lucro Total $']
    route_analysis = route_analysis.sort_values('Lucro Total $', ascending=False)
    
    print(route_analysis.head(10))

In [ ]:
# Projeção: quanto você faria em 1 mês?
if df is not None:
    print("\n💰 PROJEÇÃO DE LUCROS\n")
    print("="*60)
    
    avg_profit = df['profit_usd'].mean()
    opportunities_per_day = len(df) / (df['timestamp'].max() - df['timestamp'].min()).total_seconds() * 86400
    
    daily_profit = avg_profit * opportunities_per_day
    monthly_profit = daily_profit * 30
    
    print(f"Oportunidades por dia: {opportunities_per_day:.1f}")
    print(f"Lucro médio por operação: ${avg_profit:.2f}")
    print(f"Lucro diário estimado: ${daily_profit:.2f}")
    print(f"Lucro mensal estimado: ${monthly_profit:.2f} (R$ {monthly_profit * 5:.2f})")
    print("\n⚠️ Lembre-se: projeções não garantem resultados futuros")
    print("="*60)

## 🎯 6. Próximos Passos

### Para Modo Simulação:
1. ✅ Execute scans regulares para coletar dados
2. ✅ Analise quais rotas são mais consistentes
3. ✅ Identifique horários com mais oportunidades
4. ✅ Ajuste parâmetros (capital, lucro mínimo)

### Para Modo Live (Avançado):
1. ⚠️ Crie conta na Binance e obtenha API keys
2. ⚠️ Configure IP Whitelist
3. ⚠️ Teste com valores PEQUENOS primeiro
4. ⚠️ Monitore resultados constantemente

---

**Desenvolvido por MiniMax Agent** 🤖

⭐ Se este notebook foi útil, compartilhe!